Меринов Артем Александрович

artemmerinov@yandex.ru

In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.train = True
    """
    Basically, you can think of a module as of a something (a black box) 
    which can process `input` data and produce `ouput` data.
    This is performed by applying a `forward` function: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    Moreover, it should be able to differentiate it if is a part of chain (by chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)
    
    def backward(self, input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. (with respect to) `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    
    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
        self.output = input
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        """
        Computes the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        self.gradInput = gradOutput 
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def training(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**We define** forward and backward pass procedures for you.

In [3]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__() # this allows for calling parent ('Module') methods
                                           # from a child ('Sequential') class
        self.modules = []
        self.m_inputs = None
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)
    
    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
        """
        m_inputs = [] 
        t = input
        
        for i, mod in enumerate(self.modules):
            m_inputs.append(t)
            t = mod.forward(t)
            
        self.output = t
        self.m_inputs = m_inputs
                
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        To each module you should provide the input it saw during forward pass, 
        It is used to compute gradients. 
        Make sure that the input for `i-th` layer is the output of `module[i]` 

        """
        yinp = self.m_inputs[::-1]
        t = gradOutput
        
        for i, mod in enumerate(self.modules[::-1]):
            t = mod.backward(yinp[i], t)
                
        self.gradInput = t
        return self.gradInput

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

# Layers

The linear module computes the transformation $f(x,W,b) = Wx + b$ for a given input $x$ and parameters $W$ and $b$. The main goal of "training" our classifier will be to move from the randomly initialized $W$ and $b$ that we get by instantiating a module to the "best" $W$ and $b$ for a given data set.    

The __init__ method should take an input dimension n_in and an output dimension n_out, and should create the following instance variables:

- the n_out by n_in matrix $W$.
- the n_out vector $b$.
- the n_in vector gradInput, representing $\frac{\partial L}{\partial x}$.
- the n_out by n_in matrix gradW, representing $\frac{\partial L}{\partial W}$.
- the n_out vector gradb, representing $\frac{\partial L}{\partial b}$.

The matrix $W$ and vector $b$ should be initialized randomly. The other objects can be initialized to arrays of the correct shape filled with zeros.

The updateOutput function should update the value of the output variable and should return that output value.

The updateGradInput function should update the gradInput variable and return the new value of gradInput.

The updateGradWeight function should update both gradW and gradb and should return a tuple consisting of gradW and gradb.

In case you're rusty on your chain rule, recall that

updateGradInput: $$\frac{\partial L}{\partial x} = W^T \frac{\partial L}{\partial y}$$

accGradParameters: $$\frac{\partial L}{\partial W} = \frac{\partial L}{\partial y} x^T$$

and $$\frac{\partial L}{\partial b} = \frac{\partial L}{\partial y}$$

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [4]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out) 
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        # Your code goes here. ################################################  
        self.output = input.dot(self.W.T) + self.b
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = gradOutput.dot(self.W)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW = gradOutput.T.dot(input)
        self.gradb = np.sum(gradOutput, axis=0)
        pass
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
    
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

Output for $i$-th neuron: $$y_i = \frac{e^{z_i}}{\sum_{j=1}^{n}e^{z_j}}.$$

The derivative: $$\frac{\partial y_i}{\partial z_i} = y_i(1-y_i).$$

http://stats.stackexchange.com/questions/79454/softmax-layer-in-a-neural-network
http://math.stackexchange.com/questions/945871/derivative-of-softmax-loss-function

This one is probably the hardest but as others only takes 5 lines of code in total. 
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [5]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        # Your code goes here. ################################################
        self.output = np.exp(self.output) / np.sum(np.exp(self.output), axis=1).reshape(-1, 1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        tmp = np.multiply(self.output, gradOutput)
        self.gradInput = np.subtract(tmp, (self.output * np.sum(tmp, axis=1).reshape(-1, 1)))
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

# Activation functions

Here's the complete example for the [**Rectified Linear Unit**](https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29#ReLU) non-linearity (aka **ReLU**):
$$f(x) = \max(0,x)$$


In [6]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(0, input)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

A smooth approximation to the rectifier is the analytic function:
$$f(x) = ln ⁡(1 + e^x),$$

which is called the [**SoftPlus**](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) function.
The derivative of softplus is $$f′(x) = \frac{1}{1 + e^{−x}},$$ i.e. the logistic function.

In [7]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.log(1 + np.exp(input))
        return  self.output
    
    def updateGradInput(self, input, gradOutput):        
        self.gradInput = np.multiply(gradOutput, 1.0 / (1.0 + np.exp(-input)))
        return self.gradInput
    
    def __repr__(self):
        return "SoftPlus"

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

$$ f(x) = 
  \begin{cases} 
   x      & \text{if } x > 0 \\
   ax & \text{if } x \le 0 
  \end{cases}$$

The derivative:

$$ f'(x) = 
  \begin{cases} 
   1      & \text{if } x > 0 \\
   a & \text{if } x \le 0 
  \end{cases}$$

In [8]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
        
        self.slope = slope
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.maximum(input, self.slope * input)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.multiply(gradOutput , input > 0) + np.multiply(gradOutput, self.slope * input <= 0)
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

$$ f(x) = 
  \begin{cases} 
   x      & \text{if } x > 0 \\
   a(e^x-1) & \text{if } x \le 0 
  \end{cases}$$
  
The derivative:

$$ f'(x) = 
  \begin{cases} 
   1      & \text{if } x > 0 \\
   f(x)+a & \text{if } x \le 0 
  \end{cases} = 
  \begin{cases} 
   1      & \text{if } x > 0 \\
   ae^x & \text{if } x \le 0 
  \end{cases}$$

In [9]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()
        
        self.alpha = alpha
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        self.output = np.minimum(self.alpha * (np.exp(input) - 1), 0) + np.maximum(input, 0)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.multiply(gradOutput , input > 0) + np.multiply(np.multiply(gradOutput, 
                                                                                   self.alpha*np.exp(input)), input <= 0)
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

# Criterions

Criterions are used to score the models answers. 

In [10]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
    
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.

In [11]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target, 2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) / input.shape[0]
        return self.gradInput
    
    def __repr__(self):
        return "MSECriterion"

Your task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](https://www.kaggle.com/wiki/MultiClassLogLoss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. 

$$logloss = -\frac{1}{N}\sum_{i=1}^{N}\sum_{j=1}^{N}y_{i,j}log(p_{i,j})$$

$$logloss = −\frac{1}{N}\sum_{i=1}^{N}(y_i(log(p_i)+(1−y_{i})log(1−p_i))$$

In [12]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
        
        # Your code goes here. ################################################
        self.output = -np.sum(target * np.log(input_clamp)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
                
        # Your code goes here. ################################################
        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"

# BONUS LEVEL

### This section is **optional** and is for people who want to push their accuracy to the **max**

Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. 

This is a very cool regularizer, that can really boost your model. In fact, when you see your net is overfitting try to add more dropout.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch). When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [18]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        # Your code goes here. ################################################      
        if self.training:
            self.mask = np.random.binomial(1, self.p, input.shape)
            self.output = np.multiply(self.mask, input)
        else:
            self.output = self.p * input
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = self.mask * gradOutput
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"

Another significant recent ideaa that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement a part of the layer: mean subtraction. That is, the module should calculate mean value for every feature (every column) and subtract it.

Note, that you need to estimate the mean over the dataset to be able to predict on test examples. The right way is to create a variable which will hold smoothed mean over batches (exponential smoothing works good) and use it when forwarding test examples.

When training calculate mean as folowing:

`meanToSubtract = self.oldMean * alpha + batchMean * (1 - alpha)`

when evaluating (`self.training == False`) set $alpha = 1$

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [15]:
class BatchMeanSubtraction(Module):
    def __init__(self, alpha = 0.):
        super(BatchMeanSubtraction, self).__init__()
        
        self.alpha = alpha
        self.old_mean = 0
        
    def updateOutput(self, input):
        # Your code goes here. ################################################    
        if self.training:
            self.old_mean = self.old_mean * self.alpha + input.mean(axis=0) * (1 - self.alpha)
        self.output = input - self.old_mean
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradInput = np.subtract(gradOutput, gradOutput.mean(axis=0))
        return self.gradInput
    
    def __repr__(self):
        return "BatchMeanNormalization"